In [158]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [159]:
import pandas as pd
import re
from typing import *
import os
import copy
import numpy

In [160]:
if os.path.exists('/kaggle/input'):
    print("Kaggle")
    zebraLogicBench = pd.read_parquet('/kaggle/input/ai-connect-2025/Gridmode-00000-of-00001.parquet')
else:
    zebraLogicBench = pd.read_parquet("hf://datasets/allenai/ZebraLogicBench/grid_mode/test-00000-of-00001.parquet")
    
zebraLogicBench

,id,size,puzzle,solution,created_at
0,lgp-test-5x6-16,5*6,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Nationality', 'B...",2024-07-11T01:22:10.734298
1,lgp-test-4x4-27,4*4,"There are 4 houses, numbered 1 to 4 from left ...","{'header': ['House', 'Name', 'Occupation', 'Bo...",2024-07-11T01:22:10.732670
2,lgp-test-6x4-15,6*4,"There are 6 houses, numbered 1 to 6 from left ...","{'header': ['House', 'Name', 'Children', 'Musi...",2024-07-11T01:22:10.735021
3,lgp-test-6x5-2,6*5,"There are 6 houses, numbered 1 to 6 from left ...","{'header': ['House', 'Name', 'Mother', 'Childr...",2024-07-11T01:22:10.735170
4,lgp-test-2x2-33,2*2,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'Pet'], 'rows': [...",2024-07-11T01:22:10.730172
...,...,...,...,...,...
995,lgp-test-3x4-1,3*4,"There are 3 houses, numbered 1 to 3 from left ...","{'header': ['House', 'Name', 'Animal', 'Cigar'...",2024-07-11T01:22:10.731420
996,lgp-test-5x5-39,5*5,"There are 5 houses, numbered 1 to 5 from left ...","{'header': ['House', 'Name', 'Birthday', 'Moth...",2024-07-11T01:22:10.734197
997,lgp-test-2x2-25,2*2,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'Vacation'], 'row...",2024-07-11T01:22:10.730138
998,lgp-test-2x4-34,2*4,"There are 2 houses, numbered 1 to 2 from left ...","{'header': ['House', 'Name', 'PhoneModel', 'Oc...",2024-07-11T01:22:10.730586


# Parser

## Puzzle variables

In [161]:

class PzVariable:
    name : str
    # These are the synonyms of the variable that are used in the clues
    # Usually this just [ name ], but some use multiple different synonymes (eg: child is named ..., mother of ...)
    clues_ident : List[str]

    def __init__(self, name : str, clue_ident : List[str]):
        self.name = name
        self.clues_ident = clue_ident

    def is_for_clue_value(self, clue_ident : str):
        for c in self.clues_ident:
            if c == clue_ident:
                return True
        return False
    
    def __str__(self):
        return self.name
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, other):
        if isinstance(other, PzVariable):
            return self.name == other.name
        if isinstance(other, str):
            return self.name == other
        return False
        


class PzVariableGroup:
    name : str
    variables : List[PzVariable]
    
    def __init__(self, name : str, variables: List[PzVariable]):
        self.name = name
        self.variables = variables
        
    def __str__(self):
        t = list(map(lambda i: str(i), self.variables))
    
        return f'[{", ".join(t)}] "{self.name}"'
        # return f'[{", ".join(self.variables)}] {self.name}'
    
    def __repr__(self):
        return self.__str__()


def parse_puzzle_variable(text : str, house_count : int) -> List[PzVariableGroup]:
    
    # build variables matcher
    x = []
    for i in range(0, house_count):
        x.append('`(.*?)`')
    var_parser = ", ".join(x)
    
    
    raw_variables = re.findall(r' - (.*?): ' + var_parser, text)
    
    variables = []
    for group in raw_variables:
        group_name = group[0]
        
        # clean-up / transform the variables that they match with their usage in clues
        group_variables = []
        for v in group[1:]:
            if ("child" in group_name):
                group_variables.append(PzVariable("c." + v, [f'child is named {v}', f'mother of {v}']))
                
            elif "month" in group_name:
                if v == "jan": 
                    group_variables.append(PzVariable(v, ["january"]))
                else:
                    group_variables.append(PzVariable(v, [v]))
            
            elif "favorite color" in group_name:
                group_variables.append(PzVariable("f." + v, ["favorite color is " + v, "loves " + v]))
                
            elif "hair colors" in group_name:
                group_variables.append(PzVariable("h." + v, [v + " hair"]))
            
            # elif "keep unique animals" in group_name:
            #     l.append(f'{v} kepper')
                
            elif "hip hop" == v:
                group_variables.append(PzVariable(v, ["hip-hop"]))
                
            elif "swede" == v:
                group_variables.append(PzVariable(v, ["swedish"]))
            
            elif "ford f150" == v:
                group_variables.append(PzVariable(v, ["Ford F-150"]))
            
            elif "cat" == v: 
                group_variables.append(PzVariable(v, [" cat"])) # prevent match with vacation
                        
            else:
                v_mod = v
                if v_mod.endswith('ing'):
                    v_mod = v_mod[:-3]
                elif v_mod.endswith('s'):
                    v_mod = v_mod[:-1]
                group_variables.append(PzVariable(v, [v_mod]))
        
        variables.append(PzVariableGroup(group_name, group_variables))

    return variables

## Puzzle clues

In [162]:
class PzClue:
    clue : str
    variables : List[str]
    function : str | None
    
    def __init__(self, clue : str, vars : List[str], func : str | None):
        self.clue = clue
        self.variables = vars
        self.function = func

    def is_valid(self):
        return self.function is not None
    
    def __str__(self):
        return f'{self.variables} -> {self.function}; {self.clue}'
    
    def __repr__(self):
        return self.__str__()
    

def check_for_clue(var1, var2, regex, text):
    
    sa = regex.replace("%1", var1).replace("%2", var2)
    if re.search(sa, text, re.IGNORECASE):
        return True
    
    sb = regex.replace("%1", var2).replace("%2", var1)
    if re.search(sb, text, re.IGNORECASE):
        return True
    
    return False

def check_for_single_clue(var1, regex, text):
    
    sa = regex.replace("%1", var1)
    if re.search(sa, text, re.IGNORECASE):
        return True
    
    return False

def analyze_clue(vars, clue):
    
    if len(vars) == 1:
        if check_for_single_clue(vars[0], "%1(.*?) not in the first house", clue):
            return "not1"
        if check_for_single_clue(vars[0], "%1(.*?) not in the second house", clue):
            return "not2"
        if check_for_single_clue(vars[0], "%1(.*?) not in the third house", clue):
            return "not3"
        if check_for_single_clue(vars[0], "%1(.*?) not in the fourth house", clue):
            return "not4"
        if check_for_single_clue(vars[0], "%1(.*?) not in the fifth house", clue):
            return "not5"
        if check_for_single_clue(vars[0], "%1(.*?) not in the sixth house", clue):
            return "not6"
        if check_for_single_clue(vars[0], "%1(.*?) first house", clue):
            return "is1"
        if check_for_single_clue(vars[0], "%1(.*?) second house", clue):
            return "is2"
        if check_for_single_clue(vars[0], "%1(.*?) third house", clue):
            return "is3"
        if check_for_single_clue(vars[0], "%1(.*?) fourth house", clue):
            return "is4"
        if check_for_single_clue(vars[0], "%1(.*?) fifth house", clue):
            return "is5"
        if check_for_single_clue(vars[0], "%1(.*?) sixth house", clue):
            return "is6"
    
    if len(vars) == 2:
        if check_for_clue(vars[0], vars[1], "one house between(.*?)%1(.*?)%2", clue):
            return "oneBetween"
        if check_for_clue(vars[0], vars[1], "two houses between(.*?)%1(.*?)%2", clue):
            return "twoBetween"
        if check_for_clue(vars[0], vars[1], "%1(.*?)%2(.*?)next to each other", clue):
            return "nextTo"
        if check_for_clue(vars[0], vars[1], "%1(.*?)directly left of(.*?)%2", clue):
            return "dLeftOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)left of(.*?)%2", clue):
            return "leftOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)directly right of(.*?)%2", clue):
            return "dRightOf"
        if check_for_clue(vars[0], vars[1], "%1(.*?)right of(.*?)%2", clue):
            return "rightOf"
        # if check_for_clue(vars[0], vars[1], "%1(.*?)is (the |a )?%2", clue):
        if check_for_clue(vars[0], vars[1], "%1(.*?)is(.*?)%2", clue):
            return "equal"

    return None


def resolve_clue_var_name(variables: List[PzVariableGroup], var : str):
    for group in variables:
        for v in group.variables:
            if v.is_for_clue_value(var):
                return v.name
    return None

def analyze_clues(variables: List[PzVariableGroup], raw_clues : List[str]) -> List[PzClue]:

    # sometimes a variable contains another variables value (eg: 'child of alice' and 'alice')
    # all variables are sorted by length, each match is the removed from the clue text.
    # This ensures that none of the shorter variables can match a part from a longer variable
    all_variables = []
    for var_group in variables:
        for var in var_group.variables:
            all_variables.extend(var.clues_ident)
    all_variables = sorted(all_variables, key=len, reverse=True)
    # print(all_variables)
    
    clues = []
    for c in raw_clues:
                           
        # extract all variables used in clue
        vars = []
        test_clue = c
        for var in all_variables:
            if re.search(var, test_clue, re.IGNORECASE):
                vars.append(var)
                # test_clue = test_clue.replace(var, "")
                test_clue = re.sub(re.escape(var), '', test_clue, flags=re.IGNORECASE)
        
        # ensure variables are in the order they appear in the clue
        lower_clue = c.lower()    
        vars = sorted(vars, key=lambda s: lower_clue.find(s.lower()))       
                        
        # find the function the clue implies
        func = analyze_clue(vars, c)
        
        # resolve group name from clue variables
        vars2 = []
        for clue_var in vars:
            og_var = resolve_clue_var_name(variables, clue_var)
            if og_var is None:
                raise Exception(f"Failed to resolve all clue variables: {clue_var} from {variables}")
            vars2.append(og_var)
        
        clues.append( PzClue(c, vars2, func) )

    return clues
    

## Puzzle definition

In [163]:
class PzPuzzleDefinition:
    house_count : int
    variables : List[PzVariableGroup]
    clues : List[PzClue]
    
    def __init__(self, house_count : int, variables : List[PzVariableGroup], clues):
        self.house_count = house_count
        self.variables = variables
        self.clues = clues

    def is_valid(self):
        for c in self.clues:
            if not c.is_valid():
                return False
        return True
    
    def __repr__(self):
        s = f'Houses: {self.house_count}\n'
        s += 'Vars:\n'
        for v in self.variables:
            s += f' {v}\n'
        
        s += 'Clues:\n'
        for c in self.clues:
            s += f' {c}\n'
        
        return s


def analyze_puzzle_text(text):
    # House count
    result = re.findall(r'There are (\d+) houses, numbered 1 to \d+ from left to right', text)

    if len(result) != 1:
        raise Exception("Invalid house count")

    house_count = int(result[0])
        
    # variables
    variables = parse_puzzle_variable(text, house_count)
    
    # clues
    raw_clues = re.findall(r'\d+. (.*?)\.', text)
    clues = analyze_clues(variables, raw_clues)
    
    return PzPuzzleDefinition(house_count, variables, clues)

# CPS

## CPS Config
Keeps CPS Values, Variables and constraints

In [164]:

TVar = TypeVar('TVar')
TVal = TypeVar('TVal')

class CpsConstraint(Generic[TVal]):
    _predicates : List[Callable[[TVal, TVal], bool]]
    
    def __init__(self, predicate: Callable[[TVal, TVal], bool] = None):
        self._predicates = []
        
        if predicate is not None:
            self._predicates.append(predicate)
    
    def append(self, predicate: Callable[[TVal, TVal], bool]) -> None:
        self._predicates.append(predicate)
        
    def is_conflicting(self, a: TVal, b: TVal) -> bool:
        """
        Check if the two values a and b would conflict with any constraint
        """
        
        for p in self._predicates:
            if not p(a, b):
                return True
        return False
    
        

class CpsConfiguration(Generic[TVar, TVal]):
    _variables : List[TVar]
    _values : List[TVal]
    _constraints: Dict[TVar, Dict[TVar, CpsConstraint[TVal]]]
    
    
    def __init__(self, variables : List[str], values : List[str]):
        self._variables = variables
        self._values = values
        self._constraints = {}

    def variables(self) -> List[TVar]:
        return self._variables.copy()
    
    def values(self) -> List[TVal]:
        return self._values.copy()
    
    def _ensure_exists(self, source : TVar, target : TVar) -> None:
        if source not in self._constraints:
            self._constraints[source] = {}
        
        s = self._constraints[source]
        
        if target not in s:
            s[target] = CpsConstraint()
            
    
    def addConstraint(self, source : TVar, target : TVar, predicate : Callable[[TVal, TVal], bool]) -> None:
        """
        Add a new, one directional constraint
        """
        
        self._ensure_exists(source, target)
        
        self._constraints[source][target].append(predicate)
    
        
    def addConstraintRev(self, source : TVar, target : TVar, predicate : Callable[[TVal, TVal], bool]) -> None:
        """
        Add a new, two directional constraint.
        For the target -> source direction the parameters for the predicate are swapped, so the predicate is still called with (source, target)
        """
        
        self._ensure_exists(source, target)
        self._constraints[source][target].append(predicate)
        
        self._ensure_exists(target, source)
        self._constraints[target][source].append(lambda a, b: predicate(b, a))
    
    
    def addUnaryConstraint(self, source : TVar, predicate : Callable[[TVal], bool]) -> None:
        """
        Add a new unary constraint. 
        This is a constraint that does not have a target (eg. var must be val)
        """
        self._ensure_exists(source, None)
        
        self._constraints[source][None].append(lambda a, b: predicate(a))    
        
        
    def allNotEqual(self, variables : List[TVar]) -> None:
        """
        Constraint all variables against each other. (ie. all must have different values)
        """        
        for source in variables:
            for target in variables:
                if source == target:
                    continue
                
                self.addConstraint(source, target, lambda a, b: a != b)

                
    def notEqual(self, source : TVar, target : TVar) -> None:
        self.addConstraintRev(source, target, lambda a, b: a != b)

        
    def equal(self, source : TVar, target : TVar) -> None:
        self.addConstraintRev(source, target, lambda a, b: a == b)


    def mustBe(self, source : TVar, value : TVal) -> None:
        self.addUnaryConstraint(source, lambda a: a == value)
        
    def mustNotBe(self, source : TVar, value : TVal) -> None:
        self.addUnaryConstraint(source, lambda a: a != value)

    def get_constraints(self, variable : TVar) -> Dict[TVar, CpsConstraint[TVal] ]:
        return self._constraints[variable]




## CPS State

In [175]:

class CpsState(Generic[TVar, TVal]):
    
    _parent = None
    _variable : TVar = None
    _value : TVal = None
    _config : CpsConfiguration[TVar, TVal]    
    _assigned_cache = None
    _unassigned_cache = None
    
    
    def __init__(self, config : CpsConfiguration[TVar, TVal], parent = None, variable : TVar = None, value : TVal = None):
        self._config = config
        self._parent = parent
        self._variable = variable
        self._value = value
    
    
    def assign(self, variable, value) -> 'CpsState[TVar, TVal]':
        """
        Returns a new state with the given assignment. (A state is immutable)
        """
        if variable not in self._config.variables():
            raise Exception("Invalid variable assigned: " + str(variable), ", allowed: ", self._config.variables())
        
        if value not in self._config.values():
            raise Exception("Invalid value assigned: " + str(value), ", allowed: ", self._config.values())
        
        return CpsState(self._config, self, variable, value)
    
    
    def get_assignments(self) -> Dict[TVar, TVal]:
        """
        Get all assignments
        """
        
        if self._assigned_cache is not None:
            return self._assigned_cache.copy() # return a copy so children can add to it
        
        if self._parent is None:
            return {}
        
        if self._variable is None:
            return self._parent.get_assignments()
        
        t = self._variable, self._value
        
        t2 = self._parent.get_assignments()
        t2[self._variable] = self._value
        self._assigned_cache = t2
        return t2
    
    
    def get_assignment(self, variable: TVar):
        
        assignments = self.get_assignments()
        for var in assignments:
            val = assignments[var]
            
            if var == variable:
                return val
        return None
    
    
    def get_unassigned(self) -> List[TVar]:
        """
        Get all variables that currently have no assignment
        """
        if self._unassigned_cache is not None:
            return self._unassigned_cache.copy()
        
        # vars = self._config.variables()
        # for a in self.get_assignments():
        #     if a in vars:
        #         vars.remove(a)
            
        vars = []
        for v in self._config.variables():
            if self.get_assignment(v) is None:
                vars.append(v)
            
        self._unassigned_cache = vars
        return vars
    
    
    def get_variables(self) -> List[TVar]:
        return self._config.variables().copy()
    
    def get_values(self) -> List[TVal]:
        return self._config.values().copy()
    
    def get_constraints_for(self, variable: TVar) -> Dict[TVar, CpsConstraint[TVal]]:
        return self._config.get_constraints(variable)
    
    def is_complete(self) -> bool:
        """
        check if the CPS has assigned a value to all variables
        """
        return len(self.get_unassigned()) == 0
    
    
    def get_variables_with(self, value : TVal) -> List[TVar]:
        """
        Get all variable the value was assigned to
        """
        return [name for name, val in self.get_assignments().items() if val == value]
    
    
    def is_consistent(self) -> bool:
        
        for var in self.get_variables():
            
            # Assumes an inconsistent value was never assigned
            if self.get_assignment(var) is not None:
                continue
            
            any_consistent = False
            for val in self.get_values():
                if self.will_be_consistent(var, val):
                    any_consistent = True
            
            if not any_consistent:
                return False
        
        return True
                
    
    def will_be_consistent(self, variable: TVar, value: TVal):
        """
        Check if a variable assignment would be consistent
        """
        
        constraints = self._config.get_constraints(variable)
        
        for var in constraints:
            constraint = constraints[var] 
            
            if var is None:
                if constraint.is_conflicting(value, None):
                    return False
            else:
                otherValue = self.get_assignment(var)
                
                if otherValue is not None and constraint.is_conflicting(value, otherValue):
                    return False
                
        return True
    
    
    def get_available_values(self, variable : TVar) -> List[TVal]:
        """
        Get all values that can be assigned to variable
        """
            
        constraints = self._config.get_constraints(variable)
                   
        values = []
        
        for value in self._config.values():
        
            conflict = False
                
            for var in constraints:
                constraint = constraints[var]
                
                # if c is None it is an unary constraint
                if var is None:
                    if constraint.is_conflicting(value, None):
                        conflict = True    
                    continue
                
                otherValue = self.get_assignment(var)
                if otherValue is None: # other is not yet assigned. No check necessary
                    continue
                
                if constraint.is_conflicting(value, otherValue):
                    conflict = True
                    break
                
            if not conflict:
                values.append(value)
                
        return values
    
    def __str__(self):
        assignments = self.get_assignments()
    
        segments = []
        for val in self.get_values():
            vars = self.get_variables_with(val)
            segments.append(f'{val}: [{", ".join(map(lambda x: str(x), vars))}]')
            
        return "{" + ", ".join(segments) + "}"
        
    

# BT Search

## Core

In [166]:
from abc import abstractmethod


class BtSearchTools(Generic[TVar, TVal]):
    
    @abstractmethod
    def get_next_variable(self, state : CpsState[TVar, TVal]):
        """
        Get the next variable that should be assigned
        """
        raise NotImplementedError()
    
    @abstractmethod
    def get_values(self, state : CpsState[TVar, TVal], variable : TVar) -> List[TVal]:
        """
        """
        raise NotImplementedError()
    
    @abstractmethod
    def inference(self, state : CpsState[TVar, TVal], variable : TVar, value : TVal) -> bool:
        """
        """
        raise NotImplementedError()


class BtTraceTool(Generic[TVar, TVal]):
    
    @abstractmethod
    def assign(self, state : CpsState[TVar, TVal], variable : TVar, value : TVal) -> None:
        raise NotImplementedError()


In [167]:
class MaxDepthError(Exception):
    
    def __init__(self, state):
        super().__init__("Max depth reached")
        self.state = state        


class BtSearch(Generic[TVar, TVal]):
    _tool : BtSearchTools
    _max_depth : int
    _trace : BtTraceTool | None
    
    count : int
    result : CpsState[TVar, TVal] | None
    
    def __init__(self, tool: BtSearchTools):
        self._tool = tool
        self.count = 0
        
    
    @staticmethod
    def search(tool: BtSearchTools, initialState : CpsState[TVar, TVal], trace: BtTraceTool | None = None) -> 'BtSearch[TVar, TVal]':
        instance = BtSearch(tool)
        instance._max_depth = len(initialState.get_variables()) * len(initialState.get_values()) + 10
        instance._trace = trace
        
        instance.result = instance._search(initialState, 0)
        return instance
        
    def _search(self, state : CpsState[TVar, TVal], depth) -> CpsState[TVar, TVal] | None:
        
        if depth > self._max_depth:
            raise MaxDepthError(state)
        
        if state.is_complete():
            return state
        
        if not state.is_consistent():
            return None
        
        self.count = self.count + 1
        
        variable = self._tool.get_next_variable(state)
        if variable is None:
            return None
        
        for value in self._tool.get_values(state, variable):
            
            if state.will_be_consistent(variable, value):
                new_state = state.assign(variable, value)
                
                if self._tool.inference(new_state, variable, value):
                    
                    self._trace_assignment(state, variable, value)
                    result = self._search(new_state, depth + 1)
                    
                    if result is not None:
                        return result
        
        return None
    
    def _trace_assignment(self, state : CpsState[TVar, TVal], variable : TVar, value : TVal):
        if self._trace is not None:
            self._trace.assign(state, variable, value)
    

def get_bt_result(bt_search : BtSearch[TVar, TVal]) -> Dict[TVal, List[TVar]]:
    
    if bt_search.result is None:
        print("No result found")
        return {}
    
    print("Step count: ", bt_search.count)
    
    d = {}
    
    assignments = bt_search.result.get_assignments()
    
    for val in bt_search.result.get_values():
        
        d[val] = bt_search.result.get_variables_with(val)
        # l = []
        
        # for var in assignments:
        #     if assignments[var] == val:
        #         l.append(var)
                
        # d[val] = l
        
    return d
    

## Simple BT Search

In [168]:

class SimpleBtSearch(Generic[TVar, TVal], BtSearchTools[TVar, TVal]):
    
    def __init__(self):
        pass
    
    
    def get_next_variable(self, state : CpsState[TVar, TVal]):
        """
        Get the next variable that should be assigned
        """
        
        unassigned = state.get_unassigned()
        if len(unassigned) == 0:
            return None
        return unassigned[0]
        
    
    def get_values(self, state : CpsState[TVar, TVal], variable : TVar) -> List[TVal]:
        """
        """
        return state.get_values()
        
    
    def inference(self, state : CpsState[TVar, TVal], variable : TVar, value : TVal) -> bool:
        return True


## MRV Search

In [169]:
class MrvBtSearch(Generic[TVar, TVal], BtSearchTools[TVar, TVal]):
    
    def __init__(self):
        pass
    
    
    def get_next_variable(self, state : CpsState[TVar, TVal]):
        """
        Get the next variable that should be assigned
        """
        
        # MRV
        open_values = 10000
        variables = []
        
        for var in state.get_unassigned():
            consistent_values = 0
            
            for val in state.get_values():
                if state.will_be_consistent(var, val):
                    consistent_values = consistent_values + 1
            
            if consistent_values == 0:
                raise Exception("No consistent value found")
            
            if consistent_values > 0 and consistent_values < open_values:
                variables = [var]
                open_values = consistent_values
            
            elif consistent_values == open_values:
                variables.append(var)
                
        if len(variables) == 0:
            # something is wrong
            raise Exception("MRV selected no value")
            # return None
        
        if len(variables) == 1:
            return variables[0]
        
        
        # Gradheuristik
        variable = None
        variable_constraints = -1
        
        
        for var in variables:
            constraints = state.get_constraints_for(var)
            
            # count the number of constraints that point to unassigned variables
            c = 0
            for to in constraints:
                if to is not None and state.get_assignment(to) is None:
                    c = c + 1
                    
            if c > variable_constraints:
                variable = var
                variable_constraints = c
        
        if variable is None:
            raise Exception("Gradheuristik selected no value")
        
        return variable
        
        
    
    def get_values(self, state : CpsState[TVar, TVal], variable : TVar) -> List[TVal]:
        """
        """
        return state.get_values()
        
    
    def inference(self, state : CpsState[TVar, TVal], variable : TVar, value : TVal) -> bool:
        return True
    

In [170]:
class FileTraceTool(BtTraceTool):
    _filename : str
    
    def __init__(self, filename : str):
        self._filename = filename
        
        if os.path.exists(filename):
            os.remove(filename)
        
        
    def assign(self, state : CpsState[TVar, TVal], variable : TVar, value : TVal) -> None:
        with open(self._filename, "a") as f:
            f.write(f'{state} | {variable} | {value}\n')
        

# Puzzle solving

## CPS Setup
Creates a CPS config from a Puzzle definition

In [171]:
def next_door(a, b):
    return a == b - 1 or a == b + 1

def right_next_to(a, b):
    return (b + 1) == a 

def direct_left_of(a, b):
    # return (a - 1) == b
    return (a + 1) == b

def left_of(a, b):
    return a < b

def right_of(a, b):
    return a > b

def one_between(a, b):
    return (a + 2) == b or (a - 2) == b

def two_between(a, b):
    return (a + 3) == b or (a - 3) == b


def configure_cps(puzzle: PzPuzzleDefinition):
    if not puzzle.is_valid():
        raise Exception("Can not generate cps for invalid puzzle definition")
    
    variables = []
    for g in puzzle.variables:
        for v in g.variables:
            variables.append(v.name)
    
    values = list(range(1, puzzle.house_count + 1))
    
    config = CpsConfiguration[str, int](variables, values)
    
    for g in puzzle.variables:
        config.allNotEqual(list(map(lambda a: a.name, g.variables)))
    
    for c in puzzle.clues:
        if c.function == "equal":
            config.equal(c.variables[0], c.variables[1])
        elif c.function == "dLeftOf":
            config.addConstraintRev(c.variables[0], c.variables[1], direct_left_of)
        elif c.function == "leftOf":
            config.addConstraintRev(c.variables[0], c.variables[1], left_of)
        elif c.function == "rightOf":
            config.addConstraintRev(c.variables[0], c.variables[1], right_of)
        elif c.function == "nextTo":
            config.addConstraintRev(c.variables[0], c.variables[1], next_door)
        elif c.function == "oneBetween":
            config.addConstraintRev(c.variables[0], c.variables[1], one_between)
        elif c.function == "twoBetween":
            config.addConstraintRev(c.variables[0], c.variables[1], two_between)


        elif c.function == "not1":
            config.mustNotBe(c.variables[0], 1)    
        elif c.function == "not2":
            config.mustNotBe(c.variables[0], 2)
        elif c.function == "not3":
            config.mustNotBe(c.variables[0], 3)    
        elif c.function == "not4":
            config.mustNotBe(c.variables[0], 4)
        elif c.function == "not5":
            config.mustNotBe(c.variables[0], 5)    
        elif c.function == "not6":
            config.mustNotBe(c.variables[0], 6)       
             
        elif c.function == "is1":
            config.mustBe(c.variables[0], 1)    
        elif c.function == "is2":
            config.mustBe(c.variables[0], 2)
        elif c.function == "is3":
            config.mustBe(c.variables[0], 3)    
        elif c.function == "is4":
            config.mustBe(c.variables[0], 4)
        elif c.function == "is5":
            config.mustBe(c.variables[0], 5)    
        elif c.function == "is6":
            config.mustBe(c.variables[0], 6)
        else:
            raise Exception(f'Function not implemented: "{c.function}"')
    
    return config




In [172]:
def find_match(a, b):
    """
    Find element that is in both lists
    """
    for x in a:
        for y in b:
            if x == y:
                return x
    return None


def compare_puzzle_solutions(expected, actual):
    """
    Compare puzzle results:
    Expects: { "headers": [...], "rows": [[...], ...]}
    """

    for i in range(0, len(expected['rows'])):
        expected_row = expected["rows"][i]
        actual_row = actual["rows"][i]

        for j in range(0, len(expected_row)):
            if expected_row[j] != actual_row[j]:
                print(expected_row[j], '!=', actual_row[j])
                return False
                
    return True


def build_puzzle_solution(puzzle, result):
    """
    Takes a raw puzzle and a cps result and builds the solution dictionary
    """
    
    calculated_rows = []
    for row_index in range(0, len(puzzle['solution']["rows"])):
                
        house_assignments = result.result.get_variables_with(row_index + 1)
        
        # the variables will be returned in "random" order. 
        # But for the solution the variables must be in the order they are listed.
        # For each variable group, take the variable that is in the group
        ordered_house_assignment = []
        for g in p1.variables:
            t = find_match(map(lambda x: x.name, g.variables), house_assignments)
            if t is None:
                raise Exception("Could not find overlap in ", g.variables, house_assignments)
            ordered_house_assignment.append(t)
            
        
        row = []
        row.append(str(row_index + 1))
        
        for value in ordered_house_assignment:
            value = value
            if value.startswith('f.') or value.startswith('h.') or value.startswith('c.'):
                value = value[2:]
            row.append(value)
        
        calculated_rows.append(numpy.asarray(row, dtype=object))
    
    #display(calculated_rows)
    return {
        "header" : puzzle['solution']['header'],
        "rows": numpy.vstack(calculated_rows, dtype=object)
    }

In [185]:
failed_count = 0
failed = []

## PUZZLE 176 => max recurse error
puzzle_count = 1000

index = 0
for _, puzzle in zebraLogicBench.iterrows():
    p1 = analyze_puzzle_text(puzzle['puzzle'])
    
    if not p1.is_valid():
        print("Puzzle not valid: ", index)
        continue
    
    cpsConf = configure_cps(p1)
    baseState = CpsState(cpsConf)
    # result = BtSearch.search(SimpleBtSearch(), baseState)
    
    try:
        result = BtSearch.search(MrvBtSearch(), baseState)
    except MaxDepthError:
        print("Recursion error on: ", index)
    
    if result.result is None:
        failed.append(index)
        failed_count += 1
    
    else:
        calculated_solution = build_puzzle_solution(puzzle, result)
        c = compare_puzzle_solutions(puzzle['solution'], calculated_solution)
        
        if not c:
            print("Invalid solution", index)
    index += 1
    
print("Failed solve count:", failed_count)
print("Failed:", failed)
print("% of puzzles fail: ", failed_count / puzzle_count)

___ != 1
Invalid solution 0
___ != 1
Invalid solution 1
___ != 1
Invalid solution 2
___ != 1
Invalid solution 3
___ != 1
Invalid solution 4
___ != 1
Invalid solution 5
___ != 1
Invalid solution 6
___ != 1
Invalid solution 7
___ != 1
Invalid solution 8
___ != 1
Invalid solution 9
___ != 1
Invalid solution 10
___ != 1
Invalid solution 11
___ != 1
Invalid solution 12
___ != 1
Invalid solution 13
___ != 1
Invalid solution 14
___ != 1
Invalid solution 15
___ != 1
Invalid solution 16
___ != 1
Invalid solution 17
___ != 1
Invalid solution 18
___ != 1
Invalid solution 19
___ != 1
Invalid solution 20
___ != 1
Invalid solution 21
___ != 1
Invalid solution 22
___ != 1
Invalid solution 23
___ != 1
Invalid solution 24
___ != 1
Invalid solution 25
___ != 1
Invalid solution 26
___ != 1
Invalid solution 27
___ != 1
Invalid solution 28
___ != 1
Invalid solution 29
___ != 1
Invalid solution 30
___ != 1
Invalid solution 31
___ != 1
Invalid solution 32
___ != 1
Invalid solution 33
___ != 1
Invalid solutio

Traceback (most recent call last):
  File "C:\Users\Colin\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3672, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\Colin\AppData\Local\Temp\ipykernel_21880\2604029632.py", line 20, in <module>
    result = BtSearch.search(MrvBtSearch(), baseState)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Colin\AppData\Local\Temp\ipykernel_21880\3663577788.py", line 27, in search
    instance.result = instance._search(initialState, 0)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Colin\AppData\Local\Temp\ipykernel_21880\3663577788.py", line 55, in _search
    result = self._search(new_state, depth + 1)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Colin\AppData\Local\Temp\ipykernel_21880\3663577788.py", line 55, in _search
    result = self._search(new_state, depth + 1)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [179]:
puzzle = zebraLogicBench.iloc[211]
display(puzzle)

p1 = analyze_puzzle_text(puzzle['puzzle'])

cpsConf = configure_cps(p1)
baseState = CpsState(cpsConf)

try:
    result = None
    result = BtSearch.search(SimpleBtSearch(), baseState, FileTraceTool("trace.txt"))
    # result = BtSearch.search(MrvBtSearch(), baseState, FileTraceTool("trace.txt"))
except MaxDepthError as ex:
    print("Max depth error")
    display(str(ex.state))


if result is None or result.result is None:
    print("No solution found")
    print(puzzle['solution'])
    print(puzzle['puzzle'])
else:
   
    calculated_solution = build_puzzle_solution(puzzle, result)
    
    display(calculated_solution["rows"][0])
    
    # Compare
    expected_solution = puzzle['solution']
    print("Expected:")
    display(expected_solution)
    print("Calculated:")
    display(calculated_solution)
    
    
    print("Match:", compare_puzzle_solutions(expected_solution, calculated_solution))
    
# print(puzzle['puzzle'])

id                                               lgp-test-4x5-5
size                                                        4*5
puzzle        There are 4 houses, numbered 1 to 4 from left ...
solution      {'header': ['House', 'Name', 'Education', 'Ani...
created_at                           2024-07-11T01:22:10.732779
Name: 211, dtype: object

No solution found
{'header': array(['House', 'Name', 'Education', 'Animal', 'Pet', 'Drink'],
      dtype=object), 'rows': array([array(['___', '___', '___', '___', '___', '___'], dtype=object),
       array(['___', '___', '___', '___', '___', '___'], dtype=object),
       array(['___', '___', '___', '___', '___', '___'], dtype=object),
       array(['___', '___', '___', '___', '___', '___'], dtype=object)],
      dtype=object)}
There are 4 houses, numbered 1 to 4 from left to right, as seen from across the street. Each house is occupied by a different person. Each house has a unique attribute for each of the following characteristics:
 - Each person has a unique name: `Eric`, `Alice`, `Arnold`, `Peter`
 - Each person has a unique level of education: `master`, `high school`, `associate`, `bachelor`
 - The people keep unique animals: `bird`, `horse`, `cat`, `fish`
 - Each person has a unique type of pet: `bird`, `fish`, `dog`, `cat`
 - Each person has a unique favorite drink: `water`, `t